In [ ]:

import requests
from openpyxl import load_workbook
import pandas as pd
import time
import matplotlib.pyplot as plt
from functools import reduce

# XCAP token transfer API endpoint
base_url = "https://xcap-mainnet.explorer.xcap.network/api/v2/token-transfers"

# Set the export path
excel_path = r"C:\Users\user\Desktop\personal\DE_Assesment\DE_Assesment_Results.xlsx"

# Configs
MAX_PAGES = 50  # You can increase or decrease this
PAGE_DELAY = 0.3  # Delay between API calls in seconds


#TASK 1: Data Extraction & Processing

# Function to fetch token transfers
def fetch_all_token_transfers():
    page = 1
    all_data = []

    while page <= MAX_PAGES:
        try:
           # print(f"🚀 Fetching page {page}...")
            response = requests.get(base_url, params={"page": page}, timeout=10)
   #monitoring the igestion
            if response.status_code != 200:
                print(f" Error {response.status_code} on page {page}")
                break

            data = response.json()
            items = data.get("items", [])

            if not items:
                print(" No more data — stopping.")
                break

            all_data.extend(items)
            page += 1
            time.sleep(PAGE_DELAY)

        except requests.exceptions.RequestException as e:
            print(f" Request failed on page {page}: {e}")
            break
# report fetched data in the pipeline
    print(f" Done! Total records fetched: {len(all_data)}")
    return pd.json_normalize(all_data)

# Run the function
df = fetch_all_token_transfers()

#df.drop_duplicates(inplace=True)


fields_to_keep = [
    'transaction_hash',
    'token.symbol',
    'total.value',
    'from.hash',
    'to.hash',
    'timestamp',
    'token.exchange_rate',
    'type',
    'token.decimals'
]

# Filter columns that exist
available_fields = [col for col in fields_to_keep if col in df.columns]
df = df[available_fields]

# Export the results to xlsx 
df_fetched = df[available_fields].copy()  # Important to avoid SettingWithCopyWarning



                          #STEP 2 TRANSFORM AND CLEAN DATA
#dedup the transections
df = df.drop_duplicates(subset=fields_to_keep)

# Convert timestamp to datetime (ISO 8601 format)
#df.loc[:, 'timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce').dt.tz_localize(None)


# Clean and convert numeric fields
df.loc[:, 'total.value'] = pd.to_numeric(df['total.value'], errors='coerce').fillna(0)
df.loc[:, 'token.decimals'] = pd.to_numeric(df['token.decimals'], errors='coerce').fillna(18)

# Convert token amount to human-readable format
df.loc[:, 'normalized_value'] = df['total.value'] / (10 ** df['token.decimals'])

# Clean exchange rate and calculate USD value
df.loc[:, 'token.exchange_rate'] = pd.to_numeric(df['token.exchange_rate'], errors='coerce').fillna(0)
df.loc[:, 'usd_value'] = df['normalized_value'] * df['token.exchange_rate']

                                                # TASK 2: Compute Key Blockchain Metrics

# Total Asset Supply = Minted - Burned 
minted_total = df[df['type'] == 'token_minting']['normalized_value'].sum()
burned_total = df[df['type'] == 'token_burning']['normalized_value'].sum()
total_supply = minted_total - burned_total

# Number of Unique Tokens (by token.symbol)
unique_tokens = df['token.symbol'].nunique()

# Total Number of Transactions 
total_transactions = len(df)

#  Breakdown by Type 
minted_tokens = minted_total
burned_tokens = burned_total
transferred_tokens = df[df['type'] == 'token_transfer']['normalized_value'].sum()

# Total Transaction Volume in USD 
total_usd_volume = df['usd_value'].sum()

#  Output all key metrics
print("\n Task 2: Key Metrics")
print("1. Total Asset Supply:", total_supply)
print("2. Unique Tokens:", unique_tokens)
print("3. Total Transactions:", total_transactions)
print("4. Tokens Minted:", minted_tokens)
print("5. Tokens Burned:", burned_tokens)
print("6. Tokens Transferred:", transferred_tokens)
print("7. Total Transaction Volume (USD):", total_usd_volume)


# Prepare results as a dictionary
metrics = {
    "Metric": [
        "1. Total Asset Supply",
        "2. Unique Tokens",
        "3. Total Transactions",
        "4. Tokens Minted",
        "5. Tokens Burned",
        "6. Tokens Transferred",
        "7. Total Transaction Volume (USD)"
    ],
    "Value": [
        total_supply,
        unique_tokens,
        total_transactions,
        minted_tokens,
        burned_tokens,
        transferred_tokens,
        total_usd_volume
    ]
}

# Create DataFrame for display
metrics_df = pd.DataFrame(metrics)



# === Export both sheets ===
#with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    # Export Task 2 metrics table
  #  metrics_df.to_excel(writer, sheet_name="task2_metrics_table", index=False)

    # Export fetched records
   # df_fetched.to_excel(writer, sheet_name="Total fetched records", index=False)


#df
# TASK 3

# Ensure we are working on a fresh copy to avoid future issues
df_task3 = df.copy()

# Convert normalized_value in case it's not already computed
#df_task3['normalized_value'] = pd.to_numeric(df_task3['total.value'], errors='coerce') / (10 ** df_task3['token.decimals'])

# Step 1: Tokens Sent per Address ===
tokens_sent = df_task3[df_task3['type'] == 'token_transfer'].groupby('from.hash')['normalized_value'].sum().reset_index()
tokens_sent.columns = ['address', 'tokens_sent']

# === Step 2: Tokens Received per Address ===
tokens_received = df_task3[df_task3['type'] == 'token_transfer'].groupby('to.hash')['normalized_value'].sum().reset_index()
tokens_received.columns = ['address', 'tokens_received']

# === Step 3: Tokens Minted per Address (to.hash) ===
tokens_minted = df_task3[df_task3['type'] == 'token_minting'].groupby('to.hash')['normalized_value'].sum().reset_index()
tokens_minted.columns = ['address', 'tokens_minted']

#  Step 4: Tokens Burned per Address (from.hash) ===
tokens_burned = df_task3[df_task3['type'] == 'token_burning'].groupby('from.hash')['normalized_value'].sum().reset_index()
tokens_burned.columns = ['address', 'tokens_burned']

# === Step 5: Merge All Together to Calculate Holdings ===
from functools import reduce
dfs = [tokens_minted, tokens_burned, tokens_received, tokens_sent]
df_combined = reduce(lambda left, right: pd.merge(left, right, on='address', how='outer'), dfs).fillna(0)

# === Step 6: Calculate Final Token Holdings ===
df_combined['token_holding'] = (df_combined['tokens_minted'] - df_combined['tokens_burned']) + (df_combined['tokens_received'] - df_combined['tokens_sent'])

# === Step 7: Top 10 by Holdings ===
top10_holdings = df_combined.sort_values(by='token_holding', ascending=False).head(10)

# === Step 8: Top 10 by Sent ===
top10_sent = tokens_sent.sort_values(by='tokens_sent', ascending=False).head(10)

# === Step 9: Top 10 by Received ===
top10_received = tokens_received.sort_values(by='tokens_received', ascending=False).head(10)

# === Print Results ===
print("\n Top 10 Addresses Holding the Most Tokens:")
print(top10_holdings[['address', 'token_holding']].to_string(index=False))

print("\n Top 10 Addresses by Tokens Sent:")
print(top10_sent.to_string(index=False))

print("\n Top 10 Addresses by Tokens Received:")
print(top10_received.to_string(index=False))




# === Task 3 Summary Export Preparation ===

# Recalculate token_holding (if not already done above)
df_combined['token_holding'] = (
    df_combined['tokens_minted']
    - df_combined['tokens_burned']
    + df_combined['tokens_received']
    - df_combined['tokens_sent']
)

# Top 10s
top10_holdings = df_combined.sort_values(by='token_holding', ascending=False).head(10)
top10_sent = df_combined.sort_values(by='tokens_sent', ascending=False).head(10)[['address', 'tokens_sent']]
top10_received = df_combined.sort_values(by='tokens_received', ascending=False).head(10)[['address', 'tokens_received']]

# Totals for percentages
total_token_holding = df_combined['token_holding'].sum()
total_tokens_sent = df_combined['tokens_sent'].sum()
total_tokens_received = df_combined['tokens_received'].sum()

# Compute percentage share
def compute_percentage(df, value_col, total_val):
    df = df.copy()
    df['percentage'] = (df[value_col] / total_val * 100).round(2)
    return df

summary_holdings = compute_percentage(top10_holdings[['address', 'token_holding']], 'token_holding', total_token_holding)
summary_sent = compute_percentage(top10_sent, 'tokens_sent', total_tokens_sent)
summary_received = compute_percentage(top10_received, 'tokens_received', total_tokens_received)

# Merge all summaries
summary_report = pd.merge(summary_holdings, summary_sent, on='address', how='outer')
summary_report = pd.merge(summary_report, summary_received, on='address', how='outer')

# Rename columns
summary_report = summary_report.rename(columns={
    'token_holding': 'Token Holding',
    'percentage_x': '% of Total Holding',
    'tokens_sent': 'Tokens Sent',
    'percentage_y': '% of Total Sent',
    'tokens_received': 'Tokens Received',
    'percentage': '% of Total Received'
})

total_cleaned_records = df
df = Total_cleaned_records


#Task 4

# ========= Task 4.1: Volume per Token per Day =========
volume_per_day = df.groupby([df['timestamp'].dt.date, 'token.symbol'])['normalized_value'].sum().reset_index()
volume_per_day.columns = ['date', 'token', 'daily_volume']

# ========= Task 4.2: Cumulative Total Supply =========
df_mint = df[df['type'] == 'token_minting'].copy()
df_burn = df[df['type'] == 'token_burning'].copy()

df_mint['minted'] = df_mint['normalized_value']
df_burn['burned'] = df_burn['normalized_value']

mint_burn = pd.concat([df_mint[['timestamp', 'token.symbol', 'minted']], 
                       df_burn[['timestamp', 'token.symbol', 'burned']]], sort=False).fillna(0)
mint_burn['date'] = mint_burn['timestamp'].dt.date

supply = mint_burn.groupby(['date', 'token.symbol']).agg({'minted': 'sum', 'burned': 'sum'}).reset_index()
supply['net_minted'] = supply['minted'] - supply['burned']
supply['cumulative_supply'] = supply.groupby('token.symbol')['net_minted'].cumsum()

# ========= Task 4.3: Spike Detection =========
spike_analysis = df.groupby([df['timestamp'].dt.date, 'token.symbol', 'type'])['normalized_value'].sum().reset_index()
spike_analysis = spike_analysis.pivot(index=['timestamp', 'token.symbol'], columns='type', values='normalized_value').fillna(0)
spike_analysis.reset_index(inplace=True)

# ========= Task 4.4: Top Traded Tokens =========
traded_volume = df[df['type'] == 'token_transfer'].groupby('token.symbol')['normalized_value'].sum().sort_values(ascending=False)



task4_volume_per_day = volume_per_day
task4_cumulative_supply = supply
task4_spike_analysis = spike_analysis
task4_top_tokens = traded_volume


import streamlit as st
st.set_page_config(page_title="Token Tracker Dashboard", layout="wide")
st.title("📡 Token Analytics from XCAP API")

with st.spinner("Fetching and processing data..."):


import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates





df_summary = task3_summary_report
    df_cleaned = Total_cleaned_records
    df_trend = task4_volume_per_day   # parse_dates=['date']
    df_supply = task4_cumulative_supply # parse_dates=['date'])
    df_top = task4_top_tokens


# the chats should use in memory cleaned data for 8 hours and refresh after 8 hours
# dashboad creation

st.set_page_config(page_title="Token Analytics Dashboard", layout="wide")
st.title("📊 Token Distribution & Blockchain Trend Dashboard")

df_summary, df_cleaned, df_trend, df_supply, df_top = load_data()

# === Sidebar Filters ===
all_tokens = df_cleaned['token.symbol'].dropna().unique().tolist()
all_types = df_cleaned['type'].dropna().unique().tolist()
min_date = df_cleaned['timestamp'].min()
max_date = df_cleaned['timestamp'].max()
min_rate = float(df_cleaned['token.exchange_rate'].min())
max_rate = float(df_cleaned['token.exchange_rate'].max())

st.sidebar.header("🔍 Filter Transactions")
selected_token = st.sidebar.selectbox("Select Token Symbol", ["All"] + all_tokens)
selected_type = st.sidebar.selectbox("Select Transaction Type", ["All"] + all_types)
selected_date = st.sidebar.date_input("Filter by Date Range", [min_date, max_date])
rate_range = st.sidebar.slider("Token Exchange Rate", min_value=0.001, max_value=1.0, value=(min_rate, max_rate))

df_filtered = df_cleaned.copy()
if selected_token != "All":
    df_filtered = df_filtered[df_filtered['token.symbol'] == selected_token]
if selected_type != "All":
    df_filtered = df_filtered[df_filtered['type'] == selected_type]
df_filtered = df_filtered[
    (df_filtered['timestamp'].dt.date >= selected_date[0]) &
    (df_filtered['timestamp'].dt.date <= selected_date[1]) &
    (df_filtered['token.exchange_rate'] >= rate_range[0]) &
    (df_filtered['token.exchange_rate'] <= rate_range[1])
]

# === Summary Table ===
st.subheader("📦 Top Token Holders and Distribution Summary")
st.dataframe(df_summary.style.format({
    "Token Holding": "{:.6f}",
    "% of Total Holding": "{:.2f}%",
    "Tokens Sent": "{:.6f}",
    "% of Total Sent": "{:.2f}%",
    "Tokens Received": "{:.6f}",
    "% of Total Received": "{:.2f}%"
}))

# === Task 3 Charts ===
st.subheader("📈 Visual Breakdown by Address")
col1, col2, col3 = st.columns(3)

def labeled_barh(data, column, title, color):
    fig, ax = plt.subplots(figsize=(4, 3))
    bars = ax.barh(data['address'], data[column], color=color)
    ax.set_xlabel(column)
    ax.set_title(title, fontsize=10)
    ax.tick_params(labelsize=8)
    ax.invert_yaxis()
    for bar in bars:
        ax.text(bar.get_width(), bar.get_y() + bar.get_height() / 2, f'{bar.get_width():.2f}',
                va='center', ha='left', fontsize=8)
    return fig

with col1:
    st.pyplot(labeled_barh(df_summary, "Token Holding", "Token Holdings", "green"))

with col2:
    st.pyplot(labeled_barh(df_summary, "Tokens Sent", "Tokens Sent", "red"))

with col3:
    st.pyplot(labeled_barh(df_summary, "Tokens Received", "Tokens Received", "blue"))

# === Task 4 Charts ===
st.subheader("📉 Task 4: Blockchain Activity Trends")
col4, col5 = st.columns(2)

with col4:
    st.markdown("##### 🔄 Daily Token Transfer Volume")
    df_t4 = df_trend.copy()
    if selected_token != "All":
        df_t4 = df_t4[df_t4['token'] == selected_token]
    fig4, ax4 = plt.subplots(figsize=(6, 3))
    for token in df_t4['token'].unique():
        token_df = df_t4[df_t4['token'] == token]
        ax4.plot(token_df['date'], token_df['daily_volume'], marker='o', label=token)
        for x, y in zip(token_df['date'], token_df['daily_volume']):
            ax4.text(x, y, f"{y:.2f}", fontsize=7)
    ax4.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    ax4.tick_params(axis='x', labelrotation=45, labelsize=8)
    ax4.legend(fontsize=7)
    st.pyplot(fig4)

with col5:
    st.markdown("##### 📈 Cumulative Token Supply")
    df_sup = df_supply.copy()
    if selected_token != "All":
        df_sup = df_sup[df_sup['token.symbol'] == selected_token]
    fig5, ax5 = plt.subplots(figsize=(6, 3))
    for token in df_sup['token.symbol'].unique():
        token_df = df_sup[df_sup['token.symbol'] == token]
        ax5.plot(token_df['date'], token_df['cumulative_supply'], marker='o', label=token)
        for x, y in zip(token_df['date'], token_df['cumulative_supply']):
            ax5.text(x, y, f"{y:.2f}", fontsize=7)
    ax5.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    ax5.tick_params(axis='x', labelrotation=45, labelsize=8)
    ax5.legend(fontsize=7)
    st.pyplot(fig5)

# === Top Tokens
st.subheader("🏆 Most Transferred Tokens")
fig6, ax6 = plt.subplots(figsize=(6, 3))
bars = ax6.barh(df_top['token.symbol'], df_top['total_transferred'], color='orange')
ax6.set_xlabel("Total Transferred")
ax6.set_ylabel("Token")
ax6.tick_params(labelsize=8)
ax6.invert_yaxis()
for bar in bars:
    ax6.text(bar.get_width(), bar.get_y() + bar.get_height()/2, f"{bar.get_width():.2f}", fontsize=8, va='center', ha='left')
st.pyplot(fig6)

# === Raw Cleaned Table
st.subheader("📄 Cleaned Token Transfer Records")
st.dataframe(df_filtered.reset_index(drop=True))

st.markdown("---")
st.markdown("Made with ❤️ by Qenehelo Matjama ")
